ODEProblem:
- Functions and Jacobians: $\frac{du}{dt}$, $\frac{df}{du}$, $\frac{df}{dt}$
- Initial conditions: $u(t_{\mathrm{start}})$
- time-span: $(t_{\mathrm{start}},t_{\mathrm{final}})$

Integrator:
- `DEIntegrator.init(DEProblem, Algorithm)`
- `DEIntegrator.step()`
- `DEIntegrator.step(dt)`

Algorithm:
- Contains all information on how to step/solve


In [36]:
from abc import ABC, abstractmethod
import sys
import numpy as np
from lanre.utils import besselk1
from lanre.cosmology import ThermodynamicParticle, sself.entropy_density, sself.sqrt_gstar
from lanre.utils import plank_mass

## ODEFunction

In [70]:
class ODEFunction(ABC):
    def __init__(self):
        pass
    
    @abstractmethod
    def dudt(self, du, u, t):
        pass
    
    def dfdu(self, df, u, t):
        """
        Compute the jacobian of f = dudt, dfdu.
        
        Parameters
        ----------
        df: np.array
            Matrix to store jacobian.
        u: np.array
            Current value of u(t).
        t: float
            Current time value.
        """
        utemp = np.copy(u)
        du0 = np.zeros_like(u)
        du1 = np.zeros_like(u)
        
        self.dudt(du0, u, t)
        
        for i in range(u.shape[0]):
            ysafe = utemp[i]
            delt = np.sqrt(eps * max(1.0e-5, np.abs(ysafe)))
            utemp[i] = ysafe + delt
            self.dudt(du1, utemp, t)
            
            for j in range(u.shape[0]):
                df[j, i] = (du1[j] - du0[j]) / delt
            utemp[i] = ysafe
        
        return None
    
    def dfdt(self, df, u, t):   
        """
        Compute the gradient of f = dudt, dfdt.
        
        Parameters
        ----------
        df: np.array
            Matrix to store jacobian.
        u: np.array
            Current value of u(t).
        t: float
            Current time value.
        """
        delt = np.sqrt(sys.float_info.min * max(1.e-5, np.abs(t)))
        du = np.zeros_like(u)
        self.dudt(du, u, t)
        self.dudt(df, u, t + delt)
        # finite difference
        df = (df - du) / delt
        return None

In [45]:
class Boltzmann(ODEFunction):
    def __init__(self, mx, sigmav):
        self.sigmav = sigmav
        self.chi = ThermodynamicParticle(mx, 2.0, 1)
        
        T_start = mx
        T_end = 500.0
        logx_start = np.log(T_start)
        logx_end = np.log(T_end)
        w_start = np.array([self.chi.neq(T_start) / sself.entropy_density(T_start)])
        super(Boltzmann, self).__init__()
    
    def dudt(self, dw, w, logx):
        x = np.exp(logx)
        T = self.chi.mass() / x
        s = sself.entropy_density(T)

        weq = np.log(self.chi.neq(T) / s)
        ww = w[0]

        pf = -np.sqrt(np.pi / 45.0) * plank_mass * sself.sqrt_gstar(T) * T

        dw[0] = pf * self.sigmav * (np.exp(ww) - np.exp(2.0 * weq - ww))
    
    def dfdt(self, df, w, logx):
        x = np.exp(logx)
        T = self.chi.mass / x
        s = sself.entropy_density(T)

        weq = np.log(self.chi.neq(T) / s)
        ww = w[0]

        pf = -np.sqrt(np.pi / 45.0) * plank_mass * sself.sqrt_gstar(T) * T

        df[0,0] = pf * self.sigmav * (np.exp(ww) + np.exp(2.0 * weq - ww))
        
class Boltzmann2(ODEFunction):
    def __init__(self, mx, sigmav):
        self.sigmav = sigmav
        self.chi = ThermodynamicParticle(mx, 2.0, 1)
        
        T_start = mx
        T_end = 500.0
        logx_start = np.log(T_start)
        logx_end = np.log(T_end)
        w_start = np.array([self.chi.neq(T_start) / sself.entropy_density(T_start)])
        super(Boltzmann2, self).__init__()
    
    def dudt(self, dw, w, logx):
        x = np.exp(logx)
        T = self.chi.mass / x
        s = sself.entropy_density(T)

        weq = np.log(self.chi.neq(T) / s)
        ww = w[0]

        pf = -np.sqrt(np.pi / 45.0) * plank_mass * sself.sqrt_gstar(T) * T

        dw[0] = pf * self.sigmav * (np.exp(ww) - np.exp(2.0 * weq - ww))

In [80]:
class HarmonicOscillator(ODEFunction):
    def __init__(self, mass, k):
        self.mass = mass
        self.k = k
        super(HarmonicOscillator, self).__init__()
    
    def dudt(self, du, u, t):
        du[0] = u[1]
        du[1] = -self.k / self.mass * u[0]

## ODEProblem

In [49]:
class ODEProblem(object):
    def __init__(self, func, uinit, tstart, tend):
        self.func = func
        self.uinit = np.asarray(uinit)
        self.tstart = tstart
        self.tend = tend

## `ODEAlgorithm`

In [33]:
class ODEAlgorithm(ABC):
    def __init__(self):
        pass
    
    @abstractmethod
    def step(dt=None):
        pass

In [90]:
class DormandPrince5(ODEAlgorithm):
    def __init__(self):
        self.n = None
        self.func = None
        
        self.c2 = 0.2 
        self.c3 = 0.3 
        self.c4 = 0.8 
        self.c5 = 8.0 / 9.0 
        # Butcher a_ij's
        self.a21 = 0.2 
        self.a31 = 3.0 / 40.0 
        self.a32 = 9.0 / 40.0 
        self.a41 = 44.0 / 45.0 
        self.a42 = -56.0 / 15.0 
        self.a43 = 32.0 / 9.0 
        self.a51 = 19372.0 / 6561.0 
        self.a52 = -25360.0 / 2187.0 
        self.a53 = 64448.0 / 6561.0 
        self.a54 = -212.0 / 729.0 
        self.a61 = 9017.0 / 3168.0 
        self.a62 = -355.0 / 33.0 
        self.a63 = 46732.0 / 5247.0 
        self.a64 = 49.0 / 176.0 
        self.a65 = -5103.0 / 18656.0 
        # Butcher b_i's: y1 = y0 + h * (b1 * k1 + ... + bs * ks)
        self.a71 = 35.0 / 384.0 
        self.a73 = 500.0 / 1113.0 
        self.a74 = 125.0 / 192.0 
        self.a75 = -2187.0 / 6784.0 
        self.a76 = 11.0 / 84.0 
        # Error estimation constants: b_i^* - b_i
        self.e1 = 71.0 / 57600.0 
        self.e3 = -71.0 / 16695.0 
        self.e4 = 71.0 / 1920.0 
        self.e5 = -17253.0 / 339200.0 
        self.e6 = 22.0 / 525.0 
        self.e7 = -1.0 / 40.0 
        # Continuous output parameters
        self.d1 = -12715105075.0 / 11282082432.0 
        self.d3 = 87487479700.0 / 32700410799.0 
        self.d4 = -10690763975.0 / 1880347072.0 
        self.d5 = 701980252875.0 / 199316789632.0 
        self.d6 = -1453857185.0 / 822651844.0 
        self.d7 = 69997945.0 / 29380423.0 
        # Beta for stabilized step size control
        self.beta = 0.04 
        self.alpha = 0.2 - 0.0 * 0.75 
        # safety factor for stepsize prediction
        self.safe = 0.9 
        # Parameters for stepsize prediction
        self.minNextPrevStepRatio = 1.0 / 0.2  # < dtNext / dtPrev
        self.maxNextPrevStepRatio = 1.0 / 10.0  # > dtNext / dtPrev
        # Variables for stiffness detection
        self.maxNumStiff = 1000 
        self.numStiff = 0 
        self.numNonStiff = 0 
        self.dtLamB = 0 
        # working vectors
        self.du = None
        self.unew = None
        self.dunew = None
        self.uerr = None
        self.ustiff = None
        # runge-kutta vectors
        self.k2 = None
        self.k3 = None
        self.k4 = None
        self.k5 = None
        self.k6 = None
        # continuous output vectors
        self.rcont1 = None
        self.rcont2 = None
        self.rcont3 = None
        self.rcont4 = None
        self.rcont5 = None
        # Other
        self.dtnew = 0.0
        self.reject = False
        self.last = False
        # Counting variables
    
    def init(self, n, func):
        assert type(n) == int
        assert hasattr(func, 'dudt')
        self.n = n
        self.func = func
        
        # working vectors
        self.du = np.zeros(n)
        self.unew = np.zeros(n)
        self.dunew = np.zeros(n)
        self.uerr = np.zeros(n)
        self.ustiff = np.zeros(n)
        # runge-kutta vectors
        self.k2 = np.zeros(n)
        self.k3 = np.zeros(n)
        self.k4 = np.zeros(n)
        self.k5 = np.zeros(n)
        self.k6 = np.zeros(n)
        # continuous output vectors
        self.rcont1 = np.zeros(n)
        self.rcont2 = np.zeros(n)
        self.rcont3 = np.zeros(n)
        self.rcont4 = np.zeros(n)
        self.rcont5 = np.zeros(n)
        
    
    def compute_stages(self):
        # Computation of the 6 stages
        self.unew = self.u + self.dt * self.a21 * self.du 
        self.dudt(self.k2, self.unew, self.t + self.c2 * self.dt) 
        self.unew = self.u + self.dt * (self.a31 * self.du + self.a32 * self.k2) 
        self.dudt(self.k3, self.unew, self.t + self.c3 * self.dt) 
        self.unew = self.u + self.dt * (self.a41 * self.du + self.a42 * self.k2 + self.a43 * self.k3) 
        self.dudt(self.k4, self.unew, self.t + self.c4 * self.dt) 
        self.unew = self.u + self.dt * (self.a51 * self.du + self.a52 * self.k2 + self.a53 * self.k3 + self.a54 * self.k4) 
        self.dudt(self.k5, self.unew, self.t + self.c5 * self.dt) 
        self.ustiff = self.u + self.dt * (self.a61 * self.du + self.a62 * self.k2 + self.a63 * self.k3 + self.a64 * self.k4 + self.a65 * self.k5) 
        # The RK step
        tph = self.t + self.dt 
        self.dudt(self.k6, self.ustiff, tph) 
        self.unew = self.u + self.dt * (self.a71 * self.du + self.a73 * self.k3 + self.a74 * self.k4 + self.a75 * self.k5 + self.a76 * self.k6) 
        # Error estimate using embedded solution
        self.dudt(self.dunew, self.unew, tph) 
        self.uerr = self.dt * (self.e1 * self.du + self.e3 * self.k3 + self.e4 * self.k4 + self.e5 * self.k5 + self.e6 * self.k6 + self.e7 * self.dunew) 
        # Update counting variables
        self.numFunctionEvaluations+=1
    
    def prepare_dense(self):
        self.rcont1 = self.u
        self.rcont2 = self.unew - self.u
        self.rcont3 = self.dt * self.du - self.rcont2
        self.rcont4 = self.rcont2 - self.dt * self.dunew - self.rcont3
        self.rcont5 = self.dt * (self.d1 * self.du + self.d3 * self.k3 + self.d4 * self.k4 + 
                                 self.d5 * self.k5 + self.d6 * self.k6 + self.d7 * self.dunew)
    
    def error(self):
        err = 0.0
        for i in range(self.n):
            sk = self.abstol +  self.reltol * max(abs(self.u[i]), abs( self.unew[i]))
            sqr = self.uerr[i] / sk
            err += sqr * sqr
        return np.sqrt(err / self.n)
    
    def prepare_next_step(self,err):
        # computation of hnew
        fac11 = pow(err, self.alpha)
        # Lund-stabilization
        fac = fac11 / (self.nextPrevStepRatioPrev)**self.beta
        # we require minNextPrevStepRatio <= hnew/h <= self.maxNextPrevStepRatio
        fac = max(self.maxNextPrevStepRatio, min(self.minNextPrevStepRatio, fac / self.safe))
        self.dtnew = self.dt / fac

        if t_err <= 1.0:
            # step accepted

            self.nextPrevStepRatioPrev = max(err, 1.0E-4)
            self.num_accept+=1

            # stiffness detection
            if (self.num_accept % self.maxNumStiff == 1) or (self.numStiff > 0):
                stnum = 0.0;
                stden = 0.0;
                for i in range(self.n):
                    sqr = self.k2[i] - self.k6[i];
                    stnum += sqr * sqr;
                    sqr = self.unew[i] - self.ustiff[i];
                    stden += sqr * sqr
                if stden > 0.0:
                    self.dtLamB = self.dt * np.sqrt(stnum / stden)
                if self.dtLamB > 3.25:
                    self.numNonStiff = 0
                    self.numStiff+=1
                    if self.numStiff == 15:
                        raise RuntimeError("DormandPrince5: Equation seems to be stiff.")
                else:
                    self.numNonStiff+=1;
                    if self.numNonStiff == 6:
                        self.numStiff = 0
            if self.dense:
                self.prepare_dense()

            self.du = self.dunew;
            self.u = self.unew;
            self.tPrev = self.t;
            self.t += self.dt;

            if abs(self.dtnew) > self.dtMax:
                self.dtnew = self.tdir * self.dtmax
            if self.reject:
                self.dtnew = self.tdir * min(abs(self.dtnew), abs(self.dt))

            self.reject = False;
        else:
            # step rejected
            self.dtnew = self.dt / min(self.minNextPrevStepRatio, fac11 / self.safe);
            self.reject = true;
            if self.num_accept >= 1:
                self.num_reject+=1
            self.last = False
            
    def dense_output(self,i, t, dt):
        s = (t - self.tprev) / dt
        s1 = 1.0 - s
        return (self.rcont1[t_i] + 
                s * (self.rcont2[t_i] + 
                     s1 * (self.rcont3[t_i] + 
                           s * (self.rcont4[t_i] + 
                                s1 * self.rcont5[t_i]))))
    
    def step(self):
        while True:
            self.compute_stages()
            self.prepare_next_step(self.error())
            if not self.reject:
                break
            if self.dt <= self.t * sys.float_info.min:
                raise RuntimeError("DormandPrince5: step size too small.")
        self.num_steps += 1
    

## `ODEIntegrator`

In [35]:
class ODEIntegrator(object):
    def __init__(self, prob, alg):
        self.prob = prob
        self.alg = alg
        
        self.u = np.copy(prob.uinit)
        self.uprev = np.zeros_like(self.u)
        
        self.t = prob.tstart
        self.tprev = None
        
        self.dt = None
        self.dtPrev = None
    
    def step(dt=None):
        alg.step(dt)

## Testing

In [91]:
ho = HarmonicOscillator(1.0, 1.0)

In [92]:
prob = ODEProblem(ho, [1.0, 0.0], 0.0, 10.0)

In [93]:
alg = DormandPrince5()

In [96]:
integrator = ODEIntegrator(prob, alg)

In [94]:
alg.init(2, ho)

In [95]:
alg.step()

AttributeError: 'DormandPrince5' object has no attribute 'u'